## **SQL-Fundamentals-Farsi**

### **Calling the SQL Server Connection Function**

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def create_connection(server, database, username=None, password=None):
    """
    Create a connection to the SQL Server database using SQLAlchemy.
    
    Parameters:
    - server: str, the name of the SQL Server
    - database: str, the name of the database
    - username: str, optional, SQL Server username (for SQL authentication)
    - password: str, optional, SQL Server password (for SQL authentication)
    
    Returns:
    - engine: sqlalchemy.engine.base.Engine, the SQLAlchemy engine object or None if the connection fails
    """
    try:
        if username and password:
            connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'
        else:
            connection_string = f'mssql+pyodbc://{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'
        
        engine = create_engine(connection_string)
        
        # Test connection
        with engine.connect() as conn:
            logging.info("Connection successful!")
        
        return engine

    except SQLAlchemyError as e:
        logging.error(f"Connection failed: {e}")
        return None

***

### **Check if the database connection is successful**

In [2]:
def check_connection(engine):
    if engine:
        logging.info("Database connection established successfully.")
        return True
    else:
        logging.error("Failed to establish database connection.")
        return False

# Call the create_connection function
server = r'imsalione-pc\imsalionedb'
database = 'AdventureWorksLT2022'

engine = create_connection(server, database)

# Use the check_connection function to verify the connection
if check_connection(engine):
    # Proceed with database operations
    pass
else:
    # Handle the failed connection
    pass

2024-09-29 14:11:03,593 - INFO - Connection successful!
2024-09-29 14:11:03,595 - INFO - Database connection established successfully.


***

### **Fetching and Displaying Data from SQL Server**

In [37]:
import pandas as pd
from sqlalchemy import exc
from pathlib import Path
from prettytable import PrettyTable

# Define the directory where you want to save the report files
output_directory = Path("C:/Reports")  # Adjust the path as needed
output_directory.mkdir(parents=True, exist_ok=True)  # Ensure the directory exists

def truncate(value, width):
    return str(value)[:width] + "..." if len(str(value)) > width else str(value)

def fetch_and_display_queries(engine, queries, pnum=10, export_to_excel=False, export_to_csv=False):
    """
    Fetch data from multiple SQL queries and optionally export to Excel or CSV.
    
    Parameters:
    - engine: SQLAlchemy engine object.
    - queries: List of SQL query strings.
    - pnum: int, number of rows to display (default is 10).
    - export_to_excel: bool, whether to export data to an Excel file (default is False).
    - export_to_csv: bool, whether to export data to a CSV file (default is False).
    """
    
    if engine:
        for i, query in enumerate(queries, 1):
            print(f"Executing query {i}:\n{query}\n")
            try:
                # Load data into a pandas DataFrame
                df = pd.read_sql(query, engine)
                
                # Display the first pnum rows (or default 10 rows if pnum is not provided)
                table = PrettyTable()
                table.field_names = df.columns
                
                # Limit the width of each column to 20 characters
                for row in df.head(pnum).itertuples(index=False):
                    table.add_row([truncate(value, 25) for value in row])
                
                print(table)
                
                # Export to Excel if requested
                if export_to_excel:
                    file_path = output_directory / f"report_{i}.xlsx"
                    new_sheet_name = f"Rep_{pd.Timestamp.now().strftime('%d%m%Y%H%M%S')}"
                    
                    try:
                        with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='new') as writer:
                            df.to_excel(writer, sheet_name=new_sheet_name, index=False)
                            print(f"Data from query {i} exported to {file_path} in sheet {new_sheet_name}")
                    except Exception as e:
                        print(f"Error writing to existing Excel file: {e}")
                
                # Export to CSV if requested
                if export_to_csv:
                    csv_file_path = output_directory / f"report_{i}_{pd.Timestamp.now().strftime('%d%m%Y%H%M%S')}.csv"
                    try:
                        df.to_csv(csv_file_path, index=False)
                        print(f"Data from query {i} exported to CSV at {csv_file_path}")
                    except Exception as e:
                        print(f"Error writing to CSV file: {e}")

            except exc.SQLAlchemyError as db_error:
                print(f"Database error occurred during query {i}: {db_error}")
            except Exception as e:
                print(f"An error occurred during query {i}: {e}")
    else:
        print("No database connection.")

# Example usage:
# queries = ["SELECT * FROM table1", "SELECT * FROM table2 WHERE column = 'value'"]
# fetch_and_display_multiple_queries(engine, queries, pnum=5, export_to_excel=True)

***

### **Executing SQL Query to Fetch and Display Table Information**

In [39]:
queryes = ["SELECT TABLE_SCHEMA, TABLE_NAME FROM INFORMATION_SCHEMA.TABLES"]

fetch_and_display_queries(engine, queryes, pnum=20)

Executing query 1:
SELECT TABLE_SCHEMA, TABLE_NAME FROM INFORMATION_SCHEMA.TABLES

+--------------+------------------------------+
| TABLE_SCHEMA |          TABLE_NAME          |
+--------------+------------------------------+
|     dbo      |           ErrorLog           |
|     dbo      |         BuildVersion         |
|   SalesLT    |           Address            |
|   SalesLT    |           Customer           |
|   SalesLT    |       CustomerAddress        |
|   SalesLT    |           Product            |
|   SalesLT    |       ProductCategory        |
|   SalesLT    |      ProductDescription      |
|   SalesLT    |         ProductModel         |
|   SalesLT    | ProductModelProductDescri... |
|   SalesLT    |       SalesOrderDetail       |
|   SalesLT    |       SalesOrderHeader       |
|   SalesLT    |    vProductAndDescription    |
|   SalesLT    | vProductModelCatalogDescr... |
|   SalesLT    |      vGetAllCategories       |
+--------------+------------------------------+


In [40]:
queryes = ["""SELECT COLUMN_NAME, DATA_TYPE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'Customer';""",

"""SELECT COLUMN_NAME, DATA_TYPE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'Product';
"""]

fetch_and_display_queries(engine, queryes, pnum=20)

Executing query 1:
SELECT COLUMN_NAME, DATA_TYPE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'Customer';

+--------------+------------------+
| COLUMN_NAME  |    DATA_TYPE     |
+--------------+------------------+
|  CustomerID  |       int        |
|  NameStyle   |       bit        |
|    Title     |     nvarchar     |
|  FirstName   |     nvarchar     |
|  MiddleName  |     nvarchar     |
|   LastName   |     nvarchar     |
|    Suffix    |     nvarchar     |
| CompanyName  |     nvarchar     |
| SalesPerson  |     nvarchar     |
| EmailAddress |     nvarchar     |
|    Phone     |     nvarchar     |
| PasswordHash |     varchar      |
| PasswordSalt |     varchar      |
|   rowguid    | uniqueidentifier |
| ModifiedDate |     datetime     |
+--------------+------------------+
Executing query 2:
SELECT COLUMN_NAME, DATA_TYPE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'Product';


+------------------------+------------------+
|      COLUMN_NAME       |    DATA_TYPE     

In [11]:
query = """SELECT FirstName, LastName, 
FirstName + ' ' + LastName AS FullName 
FROM SalesLT.Customer
ORDER BY LastName DESC"""

fetch_and_display_data(engine, query, 10)

+-----------+-----------+-------------------+
| FirstName |  LastName |      FullName     |
+-----------+-----------+-------------------+
|  Caroline |  Vicknair | Caroline Vicknair |
|  Caroline |  Vicknair | Caroline Vicknair |
|   Robert  |   Vessa   |    Robert Vessa   |
|   Robert  |   Vessa   |    Robert Vessa   |
|   Wanda   |   Vernon  |    Wanda Vernon   |
|   Wanda   |   Vernon  |    Wanda Vernon   |
|    Dora   |   Verdad  |    Dora Verdad    |
|    Dora   |   Verdad  |    Dora Verdad    |
|    Raja   | Venugopal |   Raja Venugopal  |
|    Raja   | Venugopal |   Raja Venugopal  |
+-----------+-----------+-------------------+


In [92]:
query = "SELECT AVG(OrderQty) AS AverageQTY FROM SalesLT.SalesOrderDetail"

fetch_and_display_data(engine, query)

+------------+
| AverageQTY |
+------------+
|     3      |
+------------+


In [93]:
query = """SELECT OrderQty, ProductID, UnitPrice, CASE
WHEN OrderQty < 3 THEN N'کم فروش'
WHEN OrderQty >= 3 THEN N'پرفروش'
END AS SalesClass 
FROM SalesLT.SalesOrderDetail"""

fetch_and_display_data(engine, query, 15)

+----------+-----------+-----------+------------+
| OrderQty | ProductID | UnitPrice | SalesClass |
+----------+-----------+-----------+------------+
|    1     |    836    |  356.898  |  کم فروش   |
|    1     |    822    |  356.898  |  کم فروش   |
|    1     |    907    |    63.9   |  کم فروش   |
|    4     |    905    |  218.454  |   پرفروش   |
|    2     |    983    |  461.694  |  کم فروش   |
|    6     |    988    |  112.998  |   پرفروش   |
|    2     |    748    |   818.7   |  کم فروش   |
|    1     |    990    |  323.994  |  کم فروش   |
|    1     |    926    |  149.874  |  کم فروش   |
|    1     |    743    |   809.76  |  کم فروش   |
|    4     |    782    |  1376.994 |   پرفروش   |
|    2     |    918    |   158.43  |  کم فروش   |
|    4     |    780    |  1391.994 |   پرفروش   |
|    1     |    937    |   48.594  |  کم فروش   |
|    6     |    867    |   41.994  |   پرفروش   |
+----------+-----------+-----------+------------+


In [94]:
query = """SELECT COLUMN_NAME, DATA_TYPE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'Product';
"""

fetch_and_display_data(engine, query)

+-------------------+-----------+
|    COLUMN_NAME    | DATA_TYPE |
+-------------------+-----------+
|     ProductID     |    int    |
|        Name       |  nvarchar |
|   ProductNumber   |  nvarchar |
|       Color       |  nvarchar |
|    StandardCost   |   money   |
|     ListPrice     |   money   |
|        Size       |  nvarchar |
|       Weight      |  decimal  |
| ProductCategoryID |    int    |
|   ProductModelID  |    int    |
+-------------------+-----------+


In [95]:
query = """SELECT ProductNumber, StandardCost, Size, Weight FROM SalesLT.Product"""

fetch_and_display_data(engine, query)

+---------------+--------------+------+---------+
| ProductNumber | StandardCost | Size |  Weight |
+---------------+--------------+------+---------+
|   FR-R92B-58  |   1059.31    |  58  | 1016.04 |
|   FR-R92R-58  |   1059.31    |  58  | 1016.04 |
|   HL-U509-R   |   13.0863    | None |   nan   |
|    HL-U509    |   13.0863    | None |   nan   |
|   SO-B909-M   |    3.3963    |  M   |   nan   |
|   SO-B909-L   |    3.3963    |  L   |   nan   |
|   HL-U509-B   |   13.0863    | None |   nan   |
|    CA-1098    |    6.9223    | None |   nan   |
|   LJ-0192-S   |   38.4923    |  S   |   nan   |
|   LJ-0192-M   |   38.4923    |  M   |   nan   |
+---------------+--------------+------+---------+


In [96]:
query = """SELECT AVG(YEAR(ModifiedDate)) AS DateAverage,
Max(YEAR(ModifiedDate)) AS MaxYear,
Min(YEAR(ModifiedDate)) AS MinYear
FROM SalesLT.ProductDescription"""

fetch_and_display_data(engine, query)

+-------------+---------+---------+
| DateAverage | MaxYear | MinYear |
+-------------+---------+---------+
|     2007    |   2008  |   2007  |
+-------------+---------+---------+


In [97]:
query = """SELECT ProductDescriptionID, Description, ModifiedDate FROM SalesLT.ProductDescription"""

fetch_and_display_data(engine, query)

+----------------------+------------------------------+------------------------------+
| ProductDescriptionID |         Description          |         ModifiedDate         |
+----------------------+------------------------------+------------------------------+
|          3           |       Chromoly steel.        |     2007-06-01 00:00:00      |
|          4           | Aluminum alloy cups; larg... |     2007-06-01 00:00:00      |
|          5           | Aluminum alloy cups and a... |     2007-06-01 00:00:00      |
|          8           | Suitable for any type of ... |     2007-06-01 00:00:00      |
|          64          | This bike delivers a high... |     2007-06-01 00:00:00      |
|          88          | For true trail addicts.  ... |     2007-06-01 00:00:00      |
|         128          | Serious back-country ridi... | 2008-03-11 10:32:17.97300... |
|         168          | Top-of-the-line competiti... |     2007-06-01 00:00:00      |
|         170          | Suitable for any t

In [98]:
query = """SELECT MONTH(ModifiedDate) AS SaleYEAR,
DATEDIFF(MONTH, modifieddate, GETDATE())
AS YearCount FROM SalesLT.ProductDescription"""

fetch_and_display_data(engine, query)

+----------+-----------+
| SaleYEAR | YearCount |
+----------+-----------+
|    6     |    207    |
|    6     |    207    |
|    6     |    207    |
|    6     |    207    |
|    6     |    207    |
|    6     |    207    |
|    3     |    198    |
|    6     |    207    |
|    6     |    207    |
|    6     |    207    |
+----------+-----------+


In [77]:
query = """SELECT fname, lname, hire_date , DATEDIFF(YEAR, hire_date, GETDATE())
AS experience, 
CASE 
WHEN DATEDIFF(YEAR, hire_date, GETDATE()) > 30 THEN 'G1'
WHEN DATEDIFF(YEAR, hire_date, GETDATE()) <= 30 THEN 'G2'
END AS experienceClass
FROM employee"""

fetch_and_display_data(engine, query)

Database error occurred: (pyodbc.ProgrammingError) ('42S02', "[42S02] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid object name 'employee'. (208) (SQLExecDirectW)")
[SQL: SELECT fname, lname, hire_date , DATEDIFF(YEAR, hire_date, GETDATE())
AS experience, 
CASE 
WHEN DATEDIFF(YEAR, hire_date, GETDATE()) > 30 THEN 'G1'
WHEN DATEDIFF(YEAR, hire_date, GETDATE()) <= 30 THEN 'G2'
END AS experienceClass
FROM employee]
(Background on this error at: https://sqlalche.me/e/20/f405)


In [36]:
query = """SELECT * FROM sales
WHERE title_id like 'B%'"""

fetch_and_display_data(engine, query)

 id stor_id  ord_num   ord_date  qty   payterms title_id
  1    6380     6871 1994-09-14    5     Net 60   BU1032
 15    7896   QQ2299 1993-10-28   15     Net 60   BU7832
 17    7896     X999 1993-02-21   35 ON invoice   BU2075
 19    8042 423LL930 1994-09-14   10 ON invoice   BU1032
 20    8042     P723 1993-03-11   25     Net 30   BU1111


In [48]:
query = """SELECT title, type, isnull(price, 0)
AS NewPrice
FROM TITLES
ORDER BY NewPrice"""

fetch_and_display_data(engine, query)

                                               title         type  NewPrice
                  The Psychology of Computer Cooking UNDECIDED         0.00
                                       Net Etiquette popular_comp      0.00
                               The Gourmet Microwave mod_cook          2.99
                     You Can Combat Computer Stress! business          2.99
                                   Life Without Fear psychology        7.00
                 Emotional Security: A New Algorithm psychology        7.99
                                 Is Anger the Enemy? psychology       10.95
           Fifty Years in Buckingham Palace Kitchens trad_cook        11.95
Cooking with Computers: Surreptitious Balance Sheets business         11.95
                                      Sushi, Anyone? trad_cook        14.99


In [53]:
query = """SELECT TOP 3 title FROM titles
ORDER BY price DESC"""

fetch_and_display_data(engine, query)

                                                          title
                                       But Is It User Friendly?
Computer Phobic AND Non-Phobic Individuals: Behavior Variations
Onions, Leeks, and Garlic: Cooking Secrets of the Mediterranean


In [56]:
query = """SELECT TOP 3 * FROM employee
ORDER BY hire_date DESC"""

fetch_and_display_data(engine, query)

   emp_id fname minit     lname  job_id  job_lvl pub_id  hire_date
MGK44605M Matti     G Karttunen       6      220   0736 1994-05-01
KJJ92907F Karla     J Jablonski       9      170   9999 1994-03-11
PSP68661F Paula     S   Parente       8      125   1389 1994-01-19


In [59]:
query = """SELECT pub_id, price FROM titles 
WHERE price BETWEEN 7 AND 20"""

fetch_and_display_data(engine, query, 15)

pub_id  price
  1389  19.99
  1389  11.95
  1389  19.99
  0877  19.99
  1389  20.00
  0736  10.95
  0736   7.00
  0736  19.99
  0736   7.99
  0877  11.95
  0877  14.99


In [62]:
query = """select pub_id, pubdate from titles
where year(pubdate) > 1991"""

fetch_and_display_data(engine, query)

pub_id                 pubdate
  0877 2023-08-13 18:04:28.620
  1389 1994-06-12 00:00:00.000
  1389 2023-08-13 18:04:28.620


In [65]:
query = """SELECT sum(price) as SumPrice,
AVG(price) as AvgPrice,
Max(price) as MaxPrice,
Min(price) as MinPrice
FROM titles"""

fetch_and_display_data(engine, query)

 SumPrice  AvgPrice  MaxPrice  MinPrice
   236.26   14.7662     22.95      2.99


In [66]:
query = "SELECT COUNT(*) AS NumberOfAuthors FROM authors"

fetch_and_display_data(engine, query)

 NumberOfAuthors
              23


In [71]:
query = """
SELECT type, ROUND(AVG(price), 2) AS AveragePrice
FROM titles
GROUP BY type"""

fetch_and_display_data(engine, query)

        type  AveragePrice
business             13.73
mod_cook             11.49
popular_comp         21.48
psychology           13.50
trad_cook            15.96
UNDECIDED              NaN


In [7]:
query = """select count(*) as NumberOfHiredEmployee,
year(hire_date) as YearOfHiring
from employee
group by year(hire_date)
order by NumberOfHiredEmployee desc"""

fetch_and_display_data(engine, query)

 NumberOfHiredEmployee  YearOfHiring
                     8          1990
                     7          1991
                     7          1992
                     7          1993
                     7          1989
                     4          1988
                     3          1994
Data exported to C:\Reports\report.xlsx in sheet Rep_23092024152956


In [11]:
query = """
select count(*) as NumberOfHiredEmployee,
YEAR(hire_date) as YearOfHiring
from employee
group by YEAR(hire_date)
having count(*) > 5
order by NumberOfHiredEmployee desc"""

fetch_and_display_data(engine, query)

 NumberOfHiredEmployee  YearOfHiring
                     8          1990
                     7          1991
                     7          1992
                     7          1993
                     7          1989


In [27]:
query = """select count(FirstName) as CountOfPerson 
from SalesLT.Customer
"""

fetch_and_display_data(engine, query)

 CountOfPerson
           847
